In [1]:
!date
import pandas as pd
import numpy as np
import sys
sys.path.append('../GOCAM_Project/dev')

import rpy2
from rpy2.robjects.packages import importr
BiasedUrn = importr('BiasedUrn')

import utils
import enrich

pd.options.display.max_colwidth = 100
import tqdm
import os
import numba

Mon Feb 13 23:35:34 PST 2023


# Trying ncHGT on the data directly

P97 with standard method

In [2]:
df_s = enrich.enrich_wrapper('P97.csv','uniprot',method='set',show_significant=False)
np.sum(df_s['pval (uncorrected)'].values < 5e-03)

25

In [3]:
df_nc = enrich.enrich_wrapper('P97.csv','uniprot',method='ncHGT',show_significant=False)
df_nc[df_nc.title.apply(lambda x: 'PP2A' in x)]

,title,pval (uncorrected),# genes in list,#genes in gocam,shared gene products in gocam,url
69,"PI5P, PP2A and IER3 Regulate PI3K/AKT Signaling - Reactome",0.006525,8,13,"[sset:PIP5K1A-C, sset:PP2A-catalytic subunit C, sset:Activator:PI3K, sset:PI4K2A/2B, sset:Activa...",http://model.geneontology.org/R-HSA-6811558


In [4]:
df_s[df_s.title.apply(lambda x: 'RHOA GTPase cycle - Reactome' in x)]

,title,pval (uncorrected),# genes in list,#genes in gocam,shared gene products in gocam,url
31,RHOA GTPase cycle - Reactome,0.007182,2,2,"[sset:RHOA GAPs, sset:RHOA GEFs]",http://model.geneontology.org/R-HSA-8980692


In [5]:
gene_list, filtered_out_genes, filtered_list, setID2members_input_uni, setID2members_input, df_ff  = enrich.enrich_wrapper('P97.csv','uniprot',method='standard',show_significant=False,return_all =True)
len(filtered_list)

270

In [6]:
#df_ff['ncHGT'] = df_ff.apply(lambda x: do_ncHGT_new( x['# genes in list'], x['url']), axis=1)
df_ff[df_ff.title.apply(lambda x: 'PP2A' in x)]

,title,pval (uncorrected),# genes in list,#genes in gocam,shared gene products in gocam,url
198,"PI5P, PP2A and IER3 Regulate PI3K/AKT Signaling - Reactome",0.218219,8,115,"[P00533, P30154, P62714, Q9BTU6, Q99755, P35568, Q8NCE2, O14944]",http://model.geneontology.org/R-HSA-6811558


In [3]:
df_nc = enrich.enrich_wrapper('cancer.csv','Gene Symbol',method='ncHGT',show_significant=False)
df_nc

,title,pval (uncorrected),# genes in list,#genes in gocam,shared gene products in gocam,url
0,HDR through Single Strand Annealing (SSA) - Reactome,0.000000,3,7,"[P00519, Q13315, Q92889]",http://model.geneontology.org/R-HSA-5685938
1,RUNX1 regulates genes involved in megakaryocyte differentiation and platelet function - Reactome,0.000000,5,14,"[Q13951, Q09472, P15976, Q01196, sset:KMT2A-E, SETD1A,(SETD1B)]",http://model.geneontology.org/R-HSA-8936459
2,Respiratory electron transport - Reactome,0.000000,4,91,"[P09669, P21912, Q99643, O14521]",http://model.geneontology.org/R-HSA-611105
3,Stabilization of p53 - Reactome,0.000000,6,8,"[Q13315, O96017, Q00987, O15151, P04637, sset:p-S166,S188-MDM2 dimer, p-S166,S188-MDM2:MDM4]",http://model.geneontology.org/R-HSA-69541
4,GAB1 signalosome - Reactome,0.000000,3,3,"[P00533, P42336, Q06124]",http://model.geneontology.org/R-HSA-180292
...,...,...,...,...,...,...
155,RAC1 GTPase cycle - Reactome,0.884848,2,2,"[sset:RAC1 GAPs, sset:RAC1 GEFs]",http://model.geneontology.org/R-HSA-9013149
156,RAF activation - Reactome,0.920100,2,9,"[sset:RAF activating kinases, sset:RAS GEFs]",http://model.geneontology.org/R-HSA-5673000
157,G alpha (s) signalling events - Reactome,0.946717,2,11,"[sset:Gs, sset:Ligand:GPCR complexes that activate Gs]",http://model.geneontology.org/R-HSA-418555
158,G alpha (z) signalling events - Reactome,0.972745,2,10,"[sset:Ligand:GPCR complexes that activate Gi, sset:Gs]",http://model.geneontology.org/R-HSA-418597


In [6]:
# path = '../../Desktop/GOCAM/'
# datasets = {'Glover_657.csv':'MGI', 'Glover_2025.csv':'MGI'
#             }

# for dataset, symbol_type in datasets.items():
#     filename = os.path.join(path,dataset)
#     gene_list, filtered_out_genes, filtered_list, setID2members_input_uni, setID2members_input, df_display = enrich.enrich_wrapper(dataset,symbol_type, return_all = True)
#     N = len(gene_list)-len(filtered_out_genes)
#     print(N,len(filtered_list))
#     sim(N,100000)
    

134 201


100%|███████████████████████████████████| 100000/100000 [22:29<00:00, 74.10it/s]


487 572


100%|███████████████████████████████████| 100000/100000 [16:41<00:00, 99.85it/s]


In [29]:
# path = '../../Desktop/GOCAM/'
# datasets = {'gastric_cancer.csv':'ENSEMBL ID', 'cancer.csv':'Gene Symbol',#'P97.csv':'uniprot',
#             'bladder_cancer.csv':'ENSEMBL ID','parkinson.csv':'Gene Symbol',
#             }

# for dataset, symbol_type in datasets.items():
#     filename = os.path.join(path,dataset)
#     gene_list, filtered_out_genes, filtered_list, setID2members_input_uni, setID2members_input, df_display = enrich.enrich_wrapper(dataset,symbol_type, return_all = True)
#     N = len(gene_list)-len(filtered_out_genes)
#     print(N,len(filtered_list))
#     sim(N,100000)
    

45 98


100%|██████████████████████████████████| 100000/100000 [13:09<00:00, 126.67it/s]


142 216


100%|██████████████████████████████████| 100000/100000 [13:42<00:00, 121.63it/s]


28 53


100%|██████████████████████████████████| 100000/100000 [13:04<00:00, 127.53it/s]


62 87


100%|██████████████████████████████████| 100000/100000 [13:11<00:00, 126.41it/s]


In [5]:
def sim(N,size):
    setID2members = utils.csv2dict('../data/setID2members.csv')
    gocam2ID = utils.csv2dict('../data/gocam2ID_mouse.csv')
    id2g_ff = utils.csv2dict('../data/ID2gocam_mouse_ff.csv')
    Dict = utils.csv2dict('../data/ID2gocam_mouse.csv')
    genes = np.array([*id2g_ff.keys()])
    temp_gocam_keys = gocam2ID.keys()
    gc_array = {}
    for gc in temp_gocam_keys:
        gc_array[gc]=np.zeros(30)

    iterator = tqdm.tqdm(range(size), position=0, leave=True)
    for i in iterator:
        uni_list = np.random.choice(genes, size = N, replace= False)
        not_in_a_set, sets, setID2members_input_uni = enrich.get_sets(uni_list)

        filtered_out1, set_list_filtered = enrich.filter_gene_list(sets,Dict)
        filtered_out2, gene_list_filtered = enrich.filter_gene_list(uni_list, Dict) #clean gene_list to only include genes in the gocam

        filtered_list = gene_list_filtered + set_list_filtered

        counts = enrich.count_genes(filtered_list, Dict)

        for gc, k_array in gc_array.items():
            k = 0
            if gc in counts:
                k = len(counts.get(gc))
            k_array[k] +=1

            
        utils.dict2csv(gc_array, f'../sim_results/{N}x{size}.csv')


In [2]:
def get_pval(k,gc,gc_array,size):
    pval = np.sum(gc_array.get(gc)[k:])/size
    return pval

In [3]:
#utils.dict2csv(gc_array, 'sim_results_1000000.csv')
gc_array = utils.csv2dict('../sim_results/134x100000.csv')
temp = {}
for k,v in gc_array.items():
    vals = np.array([int(_[:-2]) for _ in v]) #int won't accept a string with a decimal place
    temp[k] = vals
gc_array = temp

In [5]:
pd.set_option('display.max_colwidth', None)

In [6]:
enrich.enrich_wrapper('Glover_657.csv','MGI',method='set',FDR = .15)

,title,pval (uncorrected),# entities in list,#entities in model,shared gene products in gocam,url
0,FRS-mediated FGFR2 signaling - Reactome,0.000112,5,11,"[Q8WU20, sset:FGFR2b-binding FGFs, sset:FGFR2c-binding FGFs, sset:Activated FGFR2:p-FRS, sset:Activated FGFR2]",http://model.geneontology.org/R-HSA-5654700
1,Activation of Matrix Metalloproteinases - Reactome,0.000171,6,18,"[sset:proMMP8 initial activators, sset:Trypsin, plasmin, sset:MMP7 initial activators, sset:proMMP3 initial activators, sset:proMMP1 initial activators, sset:proMMP10 activators]",http://model.geneontology.org/R-HSA-1592389
2,FRS-mediated FGFR1 signaling - Reactome,0.000288,5,13,"[Q8WU20, Q9GZV9, sset:Activated FGFR1, sset:Activated FGFR1:p-FRS, sset:FGFR1c-binding FGFs]",http://model.geneontology.org/R-HSA-5654693
3,DAP12 signaling - Reactome,0.000376,4,8,"[P43405, sset:SYK/LCK/ZAP-70, sset:p-SYK,p-BTK, sset:FYN,LCK]",http://model.geneontology.org/R-HSA-2424491
4,Signaling by ERBB2 - Reactome,0.000488,3,4,"[sset:p-Y419/420/426-N-myristoyl-SRC/FYN/YES1, sset:p-Y877-ERBB2 heterodimers, sset:ERBB2 heterodimers]",http://model.geneontology.org/R-HSA-1227986
5,MAP2K and MAPK activation - Reactome,0.000649,4,9,"[sset:RAF/MAPK scaffolds, sset:activated RAF/KSR1, sset:'activator' RAFs, sset:RAF activating kinases]",http://model.geneontology.org/R-HSA-5674135
6,Acyl chain remodelling of PI - Reactome,0.000649,4,9,"[sset:PLA2(15), sset:SYNJs,OCRL, sset:PLA2(12), sset:SYNJ/MTM(1)]",http://model.geneontology.org/R-HSA-1482922


In [7]:
enrich.enrich_wrapper('Glover_657.csv','MGI',method='ncHGT',FDR = .15)

,title,pval (uncorrected),# entities in list,#entities in model,shared gene products in gocam,url


In [8]:
enrich.enrich_wrapper('Glover_2025.csv','MGI',method='set',FDR = .05)

,title,pval (uncorrected),# entities in list,#entities in model,shared gene products in gocam,url
0,FCERI mediated Ca+2 mobilization - Reactome,0.000017,13,26,"[P43405, P16885, sset:PLCdegh, sset:SYK/BTK, sset:G-protein gamma subunit, sset:PLC-beta, sset:G(q) alpha 11,14,15,Q, sset:p-4Y-PLCG, sset:G-protein alpha (q/11/14/15), sset:ORAI dimer, sset:PPP3CA,B, sset:p-PLC gamma1,2, sset:PLCbz]",http://model.geneontology.org/R-HSA-2871809
1,Antigen activates B Cell Receptor (BCR) leading to generation of second messengers - Reactome,0.000060,12,25,"[P43405, P16885, sset:PLCdegh, sset:G-protein gamma subunit, sset:PLC-beta, sset:LYN, p-SYK, sset:G(q) alpha 11,14,15,Q, sset:G-protein alpha (q/11/14/15), sset:ORAI dimer, sset:p-PLC gamma1,2, sset:PLCbz, sset:LYN,FYN,BLK]",http://model.geneontology.org/R-HSA-983695
2,GPVI-mediated activation cascade - Reactome,0.000076,11,22,"[P06241, P43405, P16885, P42336, P30273, Q9HCN6, sset:PI3K alpha, beta, gamma, sset:p-Y348-SYK/LCK, sset:G-protein gamma subunit, sset:PI3K-catalytic subunit, sset:Activator:PI3K]",http://model.geneontology.org/R-HSA-114604
3,VEGFR2 mediated cell proliferation - Reactome,0.000151,12,27,"[P16885, sset:p-PKC, sset:PLCdegh, sset:G-protein gamma subunit, sset:PLC-beta, sset:PI3K-catalytic subunit, sset:G(q) alpha 11,14,15,Q, sset:G-protein alpha (q/11/14/15), sset:ORAI dimer, sset:p-PKCA,p-PKCB,p-PKCZ,p-PKCD, sset:p-PLC gamma1,2, sset:PLCbz]",http://model.geneontology.org/R-HSA-5218921


In [9]:
enrich.enrich_wrapper('Glover_2025.csv','MGI',method='ncHGT',FDR = .05)

,title,pval (uncorrected),# entities in list,#entities in model,shared gene products in gocam,url
0,Regulation of PTEN stability and activity - Reactome,0.000000,9,56,"[P42685, P28070, P28066, P14373, P67870, Q06323, sset:USP13,OTUD3, sset:CSNK2(A1:A1/A1:A2/A2:A2), sset:NEDD4,STUB1,WWP2 and XIAP]",http://model.geneontology.org/R-HSA-8948751
1,Regulation of expression of SLITs and ROBOs - Reactome,0.000000,3,51,"[P28070, P28066, Q06323]",http://model.geneontology.org/R-HSA-9010553
2,CDK-mediated phosphorylation and removal of Cdc6 - Reactome,0.000000,4,65,"[P28070, P28066, P60006, Q06323]",http://model.geneontology.org/R-HSA-69017
3,APC/C:Cdc20 mediated degradation of Securin - Reactome,0.000000,4,62,"[P28070, P28066, P60006, Q06323]",http://model.geneontology.org/R-HSA-174154
4,Choline catabolism - Reactome,0.000000,4,8,"[Q8NE62, Q9UI17, Q8WWI5, sset:CTL1-5]",http://model.geneontology.org/R-HSA-6798163
5,PI-3K cascade:FGFR2 - Reactome,0.000000,4,7,"[Q8WU20, Q13480, P42336, sset:Activated FGFR2]",http://model.geneontology.org/R-HSA-5654695
6,Ubiquitin Mediated Degradation of Phosphorylated Cdc25A - Reactome,0.000000,3,49,"[P28070, P28066, Q06323]",http://model.geneontology.org/R-HSA-69601
7,GSK3B and BTRC:CUL1-mediated-degradation of NFE2L2 - Reactome,0.000000,3,44,"[P28070, P28066, Q06323]",http://model.geneontology.org/R-HSA-9762114
8,KEAP1-NFE2L2 pathway - Reactome,0.000000,5,55,"[P28070, P28066, P67870, Q06323, sset:CSNK2(A1:A1/A1:A2/A2:A2)]",http://model.geneontology.org/R-HSA-9755511
9,Regulation of RUNX2 expression and activity - Reactome,0.000000,3,54,"[P28070, P28066, Q06323]",http://model.geneontology.org/R-HSA-8939902


In [30]:
setID2members = utils.csv2dict('../data/setID2members.csv')
gocam2ID = utils.csv2dict('../data/gocam2ID_mouse.csv')
id2g_ff = utils.csv2dict('../data/ID2gocam_mouse_ff.csv')
Dict = utils.csv2dict('../data/ID2gocam_mouse.csv')

df_s = enrich.enrich_wrapper('Glover_657.csv','MGI',method='set',show_significant=False)

size = 100000#1000000
FDR = .9
show_significant = True
background_num_gocams = len(gocam2ID)
df_sim = df_s.copy()
df_sim['pval (uncorrected)']=df_sim.apply(lambda x: get_pval(x['# genes in list'], x['url'],gc_array,size), axis =1)

df_sim.sort_values('pval (uncorrected)',inplace=True)
df_sim.reset_index(drop=True, inplace=True)
df_sim['FDR_val'] = (df_sim.index+1)*FDR/background_num_gocams
df_sim['Less_than'] = (df_sim['pval (uncorrected)'] < df_sim['FDR_val'])
index = df_sim.Less_than.where(df_sim.Less_than==True).last_valid_index()
df_significant = df_sim
if (show_significant):
    df_significant = df_sim.loc[0:index].copy()
    if index == None:
        df_significant = pd.DataFrame(columns =['url', 'pval (uncorrected)', '# genes in list','#genes in gocam','shared gene products in gocam'])
df_sim = df_significant[['url','pval (uncorrected)', '# genes in list', '#genes in gocam','shared gene products in gocam']].copy()
df_sim

,url,pval (uncorrected),# genes in list,#genes in gocam,shared gene products in gocam
0,http://model.geneontology.org/R-HSA-5621480,0.00126,2,2,"[P43405, sset:FYN, LYN]"
1,http://model.geneontology.org/R-HSA-8964058,0.00192,2,3,"[Q9Y5X9, P04180]"
2,http://model.geneontology.org/R-HSA-373753,0.00204,2,3,"[P06241, O60500]"


In [18]:
df17 = pd.read_csv('../../Desktop/Network_data/mouse_samples_insilicoPlasmid_allreps_SGwideMin250counts.txt.sgrna_summary_SGwideMin250counts.txt',sep='\t')
df17#['Uniprot.ID']

,Genes,MeanDSD,NrSGMZ,NrSGMZ2
0,Prp2,36.026387,3,1
1,Sprr2d,32.720790,1,1
2,Rnf208,31.193251,1,0
3,Kctd17,29.532982,1,1
4,Raf1,26.905317,1,1
...,...,...,...,...
17269,Rabgef1,0.577350,2,0
17270,Rtn4,0.577350,2,0
17271,Slc4a3,0.577350,1,0
17272,Sumf2,0.577350,3,0


In [4]:
df650 = pd.read_csv('../../Desktop/Network_data/mouse_samples_insilicoPlasmid_allreps_Min1000counts_Full.txt.sgrna_summary_Min1000counts_Full.txt',sep='\t')
df650['MGI.Gene.Marker.ID'].to_csv('../../Desktop/GOCAM/Glover_657.csv')

In [3]:
df2000 = pd.read_csv('../../Desktop/Network_data/mouse_samples_insilicoPlasmid_allreps_Min500counts_Full.txt.sgrna_summary_Min500counts_Full.txt',sep='\t')
df2000['MGI.Gene.Marker.ID'].to_csv('../../Desktop/GOCAM/Glover_2025.csv')

In [22]:
df3000 = pd.read_csv('../../Desktop/Network_data/mouse_samples_insilicoPlasmid_2reps_1000counts_SGwide.txt.sgrna_summary_SGwide.txt',sep='\t')
df3000#['Uniprot.ID']

,Genes,SG1,SG2,SG3,SG4,SG5,SG6,SG7,SG8,SG9,SG10
0,4921509C19Rik,142.938333,1519.133333,2038.133333,2.232633,1493.526667,41.156000,4097.500000,1061.446667,369.130000,371.886667
1,Smim14,0.000000,2886.600000,1498.473333,0.000000,0.000000,0.000000,1769.170000,0.000000,1213.130000,3846.933333
2,Rab10,0.000000,1036.010667,8584.766667,0.000000,0.000000,128.536667,1194.200000,1769.366667,3721.866667,0.000000
3,Gm7257,0.000000,0.000000,4.146333,1982.933333,0.000000,1163.906667,0.000000,1423.333333,1709.266667,5928.133333
4,2310050C09Rik,0.000000,3418.766667,266.968990,4270.266667,1746.363333,0.000000,0.000000,1251.600000,0.000000,2188.133333
...,...,...,...,...,...,...,...,...,...,...,...
3023,Erlec1,0.000000,0.000000,0.000000,0.000000,0.000000,342.783333,1497.190000,0.000000,1016.283333,0.000000
3024,Ascl4,27.655667,812.670000,336.928000,1021.626667,0.000000,0.000000,0.000000,0.000000,1373.126667,53.963000
3025,Srpx2,0.000000,0.000000,33.206233,0.000000,1634.873333,149.133333,0.000000,0.000000,0.000000,1149.240000
3026,Rhox13,0.000000,0.000000,0.000000,0.000000,472.523333,1254.523333,1329.306667,0.000000,0.000000,0.000000


In [25]:
df2900 = pd.read_csv('../../Desktop/Network_data/mouse_human_samples_insilicoPlasmid_2reps_1000counts_SGwide.txt.sgrna_summary_SGwide.csv')
df2900#['Uniprot.ID']

,Unnamed: 0,gene.names,PI,Number.of.sgRNAs,Organism,description,Human,Species,SG1,SG2,SG3,SG4,SG5,SG6,SG7,SG8,SG9,SG10
0,3504,Mar-04,MGI:2683550,4.0,Mus musculus,membrane-associated ring finger (C3HC4) 4,Mar-04,H. sapiens,0.000000,22.541667,0.000000,11161.600000,141.281233,0.000000,114.166667,1158.786667,1372.106667,487.156667
1,5110,Sep-07,MGI:1335094,2.0,Mus musculus,septin 7,SEPTIN7,H. sapiens,0.000000,77.643333,1079.613333,0.000000,62.557333,1170.413333,0.318947,0.000000,0.000000,0.000000
2,6654,1100001G20Rik,MGI:1913357,2.0,Mus musculus,WAP four-disulfide core domain 21,WFDC21P,H. sapiens,0.000000,0.000000,0.000000,11953.166670,513.110000,0.000000,508.271000,0.000000,845.156667,4325.300000
3,3,1110008F13Rik,MGI:1914638,3.0,Mus musculus,"Rab5if, RAB5 interacting factor",RAB5IF,H. sapiens,0.000000,1271.223333,3200.633333,13.896000,0.000000,0.000000,0.000000,1438.480000,9.218667,0.000000
4,7,1110065P20Rik,MGI:1916170,4.0,Mus musculus,RIKEN cDNA 1110065P20 gene,C1orf122,H. sapiens,0.000000,3063.833333,63.428947,0.000000,0.000000,2760.766667,525.320000,0.000000,0.000000,3842.466667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2898,6814,Nmrk1,MGI:2147434,3.0,Mus musculus,Nicotinamide riboside kinase 1,NMRK1,H. sapiens,3811.433333,0.000000,0.000000,0.000000,4032.200000,0.000000,2.481900,0.000000,2328.100000,0.000000
2899,6818,Nup62-il4i1,MGI:3534327,2.0,Mus musculus,RIKEN clone 4930569A11,NaN,H. sapiens,4033.466667,0.000000,0.000000,0.000000,1278.213333,261.336667,0.000000,0.000000,0.000000,0.000000
2900,6728,Dhrs2,MGI:1918662,2.0,Mus musculus,dehydrogenase/reductase member 2,DHRS2,H. sapiens,4495.400000,0.000000,133.924667,4213.066667,0.000000,90.709333,0.000000,0.000000,14.033667,630.223333
2901,6678,4930504O13Rik,MGI:2685370,2.0,Mus musculus,LY6/PLAUR domain containing 9,LYPD9,H. sapiens,4599.866667,0.000000,0.000000,0.318947,1224.050000,0.000000,48.062000,854.539000,22.700333,0.000000
